In [1]:
import numpy as np
import keras
from keras.layers import Dense, Input, Add
from keras.models import Sequential, Model, load_model

from Source import BEC

Using TensorFlow backend.


In [2]:
n_power=7
CodeRate=0.5
N_length=int(2**n_power)
msg_length=int(CodeRate*N_length)
frozen_length=N_length-msg_length
TotalNumMessage=4
codeNum = 10**6 
test_codeNum = 10**3

Prob_start=0.15
Prob_delta=0.05
BEC_count=8
BECs=np.arange(Prob_start,Prob_start+BEC_count*Prob_delta,Prob_delta)

In [3]:
G_All=BEC.AllGeneratorMatrixPolarCode(n_power) ###생성행렬생성

In [4]:
(Z_BEC,Z_decending_index_BEC)=BEC.BhanttacharyyaParameterBEC(N_length, n_power)

In [5]:
SelectedBitsPositions = Z_decending_index_BEC[frozen_length:]
SelectedBitsPositions.sort()

In [6]:
FrozenBitsPositions = Z_decending_index_BEC[:frozen_length]
FrozenBitsPositions.sort()

In [7]:
msg, codeword = BEC.Encoding(codeNum, N_length, n_power, msg_length, SelectedBitsPositions, G_All[n_power]) 
ReceviedBEC = BEC.ReceviedCode(0.25, codeNum, N_length, codeword)

In [10]:
test_msg, test_codeword = BEC.Encoding(test_codeNum, N_length, n_power, msg_length, SelectedBitsPositions, 
                                       G_All[n_power]) 
test_ReceviedBEC = BEC.ReceviedCode(0.25, test_codeNum, N_length, test_codeword)

In [13]:
input_layer = Input(shape=(128,))
line1_Dense1 = Dense(128*1, activation='elu')(input_layer)
line1_Dense2 = Dense(128*1, activation='elu')(line1_Dense1)         
line1_Dense3 = Dense(128*1, activation='elu')(line1_Dense2)
line1_output_layer = Dense(128, activation='tanh')(line1_Dense3)

line2_Dense1 = Dense(128*2, activation='elu')(input_layer)
line2_Dense2 = Dense(128*4, activation='elu')(line2_Dense1)         
line2_Dense3 = Dense(128*2, activation='elu')(line2_Dense2)
line2_output_layer = Dense(128, activation='tanh')(line2_Dense3)

line3_Dense1 = Dense(128*3, activation='elu')(input_layer)
line3_Dense2 = Dense(128*9, activation='elu')(line3_Dense1)         
line3_Dense3 = Dense(128*3, activation='elu')(line3_Dense2)
line3_output_layer = Dense(128, activation='tanh')(line3_Dense3)

line4_Dense1 = Dense(128*4, activation='elu')(input_layer)
line4_Dense2 = Dense(128*16, activation='elu')(line4_Dense1)         
line4_Dense3 = Dense(128*4, activation='elu')(line4_Dense2)
line4_output_layer = Dense(128, activation='tanh')(line4_Dense3)

line5_Dense1 = Dense(128*5, activation='elu')(input_layer)
line5_Dense2 = Dense(128*25, activation='elu')(line5_Dense1)         
line5_Dense3 = Dense(128*5, activation='elu')(line5_Dense2)
line5_output_layer = Dense(128, activation='tanh')(line5_Dense3)

line6_Dense1 = Dense(128*6, activation='elu')(input_layer)
line6_Dense2 = Dense(128*36, activation='elu')(line6_Dense1)         
line6_Dense3 = Dense(128*6, activation='elu')(line6_Dense2)
line6_output_layer = Dense(128, activation='tanh')(line6_Dense3)

add_layer = Add()([line1_output_layer, line2_output_layer, line3_output_layer, line4_output_layer, 
                   line5_output_layer, line6_output_layer])

output_layer = Dense(128, activation='linear')(add_layer)

model = Model(inputs = input_layer, outputs=output_layer)

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

In [15]:
model.fit(ReceviedBEC, codeword, 
          epochs=100,
          batch_size=10000,
          shuffle=True,
          validation_data=(test_ReceviedBEC, test_codeword))

1000000/1000000 [==============================] - 18s 18us/step - loss: 0.0649 - acc: 0.0196 - val_loss: 0.0660 - val_acc: 0.0220
Epoch 31/100
1000000/1000000 [==============================] - 18s 18us/step - loss: 0.0649 - acc: 0.0203 - val_loss: 0.0658 - val_acc: 0.0230
Epoch 32/100
1000000/1000000 [==============================] - 17s 17us/step - loss: 0.0648 - acc: 0.0203 - val_loss: 0.0658 - val_acc: 0.0140
Epoch 33/100
1000000/1000000 [==============================] - 17s 17us/step - loss: 0.0648 - acc: 0.0203 - val_loss: 0.0658 - val_acc: 0.0110
Epoch 34/100
1000000/1000000 [==============================] - 17s 17us/step - loss: 0.0647 - acc: 0.0207 - val_loss: 0.0659 - val_acc: 0.0210
Epoch 35/100
1000000/1000000 [==============================] - 17s 17us/step - loss: 0.0647 - acc: 0.0206 - val_loss: 0.0658 - val_acc: 0.0240
Epoch 36/100
1000000/1000000 [==============================] - 17s 17us/step - loss: 0.0647 - acc: 0.0202 - val_loss: 0.0658 - val_acc: 0.0170
Epoch

In [18]:
model.save('Model/model_BEC_p6.h5')